In [0]:
import os, sys, json, glob
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications

In [0]:
DVC_URL = 'http://files.fast.ai/data/dogscats.zip'
TPU_ADDR = f'grpc://{os.environ["COLAB_TPU_ADDR"]}'
IMG_SIZE = 150

In [0]:
def download_data(url:str, cache_dir:str='.'):
  fn = url.split('/')[-1]
  tf.keras.utils.get_file(fn, origin=url, extract=True, cache_dir=cache_dir)

In [0]:
download_data(DVC_URL)

In [0]:
DATA_DIR = './datasets/dogscats/'

In [0]:
top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = DATA_DIR + 'train/'
validation_data_dir = DATA_DIR + 'valid/'
nb_train_samples = 23000
nb_validation_samples = 2000
epochs = 50
batch_size = 128

In [0]:
train_datagen = ImageDataGenerator(rescale=1.0/255)
valid_datagen = ImageDataGenerator(rescale=1.0/255)

In [8]:
vgg16 = applications.VGG16(include_top=False,
   input_shape=(IMG_SIZE, IMG_SIZE,3))

for l in vgg16.layers:
  l.trainable = False

h = vgg16.output
h = Flatten()(h)
h = Dense(256, activation='relu')(h)
h = Dropout(0.5)(h)
h = Dense(1, activation='sigmoid')(h)

model = tf.keras.models.Model(vgg16.input, h)
model.compile(optimizer=tf.train.AdamOptimizer(), loss='binary_crossentropy', 
  metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDR)
    )
)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.92.239.202:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 15713806325633069451)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 1638909662840706597)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 10293082847027371014)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU

In [10]:
train_generator = train_datagen.flow_from_directory(
  train_data_dir,
  target_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size,
  class_mode='binary')

validation_generator = valid_datagen.flow_from_directory(
  validation_data_dir,
  target_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size,
  class_mode='binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [11]:
model.fit_generator(
  train_generator,
  steps_per_epoch=nb_train_samples // batch_size,
  epochs=50,
  validation_data=validation_generator,
  validation_steps=nb_validation_samples // batch_size)

# model.save_weights(top_model_weights_path)

Epoch 1/50
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 150, 150, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 1), dtype=tf.float32, name='dense_1_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 17.814032077789307 secs
INFO:tensorflow:Setting weights on TPU model.
106/180 [================>.............] - ETA: 49s - loss: 0.3464 - acc: 0.8550INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(11,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(11, 150, 150, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(11, 1), dtype=tf.float32, name='dense_1_target_30')]